# Hit API

In [7]:
%env CALTRAIN_API_KEY=66114e41-7929-4a61-a9c5-96cff817e7df

env: CALTRAIN_API_KEY=66114e41-7929-4a61-a9c5-96cff817e7df


In [8]:
from caltrain.constants import (
    ID2NAME,
    RWC_CALTRAIN_STOP_ID,
    MY_TRAIN_TIME_COLS,
    API_TRAIN_TIME_COLS,
    MAP_TO_AVAILABLE_STATION,
    MINUTES_FROM_PRE_SF_STOP_NORTHWARD,
    MINUTES_FROM_PRE_SF_STOP_SOUTHWARD
)

from caltrain.rwc_sf_trains import RwcSfTrains
from caltrain.util import (
    replace_colnames,
    iso_to_timestamp,
    convert_time_str_to_local_tz_timestamp,
    format_for_display,
    one_vehicle_activity_to_stops_with_vehicle_id,
)

In [20]:
{os.environ.pop(k) for k in os.environ.keys() if 'PROXY' in k}

set()

In [18]:
# self = RwcSfTrains('north')
self = RwcSfTrains('south')
self.fetch_data()

In [23]:
self.departures_response

{'redwood_city': {'ServiceDelivery': {'ResponseTimestamp': '2024-09-16T16:45:19Z',
   'ProducerRef': 'CT',
   'Status': True,
   'StopMonitoringDelivery': {'version': '1.4',
    'ResponseTimestamp': '2024-09-16T16:45:19Z',
    'Status': True,
    'MonitoredStopVisit': [{'RecordedAtTime': '1970-01-01T00:00:00Z',
      'MonitoringRef': '70241',
      'MonitoredVehicleJourney': {'LineRef': 'L5',
       'DirectionRef': 'N',
       'FramedVehicleJourneyRef': {'DataFrameRef': '2024-09-16',
        'DatedVehicleJourneyRef': '503'},
       'PublishedLineName': 'LTD 5',
       'OperatorRef': 'CT',
       'OriginRef': '70261',
       'OriginName': 'San Jose Diridon Caltrain Station Northbound',
       'DestinationRef': '70011',
       'DestinationName': 'San Francisco Caltrain Station Northbound',
       'Monitored': True,
       'InCongestion': None,
       'VehicleLocation': {'Longitude': '', 'Latitude': ''},
       'Bearing': None,
       'Occupancy': None,
       'VehicleRef': None,
       '

In [17]:
self.real_time_response

{'Siri': {'ServiceDelivery': {'ResponseTimestamp': '2024-09-16T16:43:49Z',
   'ProducerRef': 'CT',
   'Status': True,
   'VehicleMonitoringDelivery': {'version': '1.4',
    'ResponseTimestamp': '2024-09-16T16:43:49Z',
    'VehicleActivity': [{'RecordedAtTime': '2024-09-16T16:43:41Z',
      'ValidUntilTime': '',
      'MonitoredVehicleJourney': {'LineRef': 'L1',
       'DirectionRef': 'S',
       'FramedVehicleJourneyRef': {'DataFrameRef': '2024-09-16',
        'DatedVehicleJourneyRef': '110'},
       'PublishedLineName': 'Local',
       'OperatorRef': 'CT',
       'OriginRef': '70012',
       'OriginName': 'San Francisco Caltrain Station Southbound',
       'DestinationRef': '70272',
       'DestinationName': 'Tamien Caltrain Station Southbound',
       'Monitored': True,
       'InCongestion': None,
       'VehicleLocation': {'Longitude': '-122.14045',
        'Latitude': '37.4283485'},
       'Bearing': None,
       'Occupancy': None,
       'VehicleRef': '110',
       'MonitoredCall

In [134]:

north = RwcSfTrains('north')
south = RwcSfTrains('south')

In [12]:
from caltrain.constants import PRE_SF_STOP_FROM_LINE_TYPE, MINUTES_TO_SF_LINE_TYPE

In [81]:
self.get_vehicle_onward_stops()

,stop_name,scheduled_departure,expected_departure,vehicle_id,line_type
0,22nd Street Caltrain Station,2023-07-06 15:16:00-07:00,2023-07-06 15:16:45-07:00,408,LTD 4
1,San Bruno Caltrain Station,2023-07-06 15:27:00-07:00,2023-07-06 15:27:30-07:00,408,LTD 4
2,Millbrae Caltrain Station,2023-07-06 15:32:00-07:00,2023-07-06 15:32:30-07:00,408,LTD 4
3,Burlingame Caltrain Station,2023-07-06 15:37:00-07:00,2023-07-06 15:37:45-07:00,408,LTD 4
4,San Mateo Caltrain Station,2023-07-06 15:41:00-07:00,2023-07-06 15:41:15-07:00,408,LTD 4
5,San Carlos Caltrain Station,2023-07-06 15:49:00-07:00,2023-07-06 15:49:00-07:00,408,LTD 4
6,Redwood City Caltrain Station,2023-07-06 15:53:00-07:00,2023-07-06 15:53:15-07:00,408,LTD 4
7,Palo Alto Caltrain Station,2023-07-06 16:00:00-07:00,2023-07-06 16:00:45-07:00,408,LTD 4
8,Mountain View Caltrain Station,2023-07-06 16:08:00-07:00,2023-07-06 16:08:30-07:00,408,LTD 4
9,Sunnyvale Caltrain Station,2023-07-06 16:13:00-07:00,2023-07-06 16:13:45-07:00,408,LTD 4


In [25]:
self.all_rwc_trains_and_onward_stops().data

trains {'410', '124', '708', '308'} not in the realtime response :thunk:


,stop name,scheduled departure,expected departure,time late,stop name onward stop,scheduled departure onward stop,expected departure onward stop,line type,trip duration from origin
0,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,22nd Street Caltrain Station,2023-07-06 15:16:00-07:00,2023-07-06 15:16:45-07:00,LTD 4,0 days 00:05:45
1,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,San Bruno Caltrain Station,2023-07-06 15:27:00-07:00,2023-07-06 15:27:30-07:00,LTD 4,0 days 00:16:30
2,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,Millbrae Caltrain Station,2023-07-06 15:32:00-07:00,2023-07-06 15:32:30-07:00,LTD 4,0 days 00:21:30
3,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,Burlingame Caltrain Station,2023-07-06 15:37:00-07:00,2023-07-06 15:37:45-07:00,LTD 4,0 days 00:26:45
4,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,San Mateo Caltrain Station,2023-07-06 15:41:00-07:00,2023-07-06 15:41:15-07:00,LTD 4,0 days 00:30:15
5,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,San Carlos Caltrain Station,2023-07-06 15:49:00-07:00,2023-07-06 15:49:00-07:00,LTD 4,0 days 00:38:00
6,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,Redwood City Caltrain Station,2023-07-06 15:53:00-07:00,2023-07-06 15:53:15-07:00,LTD 4,0 days 00:42:15
7,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,Palo Alto Caltrain Station,2023-07-06 16:00:00-07:00,2023-07-06 16:00:45-07:00,LTD 4,0 days 00:49:45
8,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,Mountain View Caltrain Station,2023-07-06 16:08:00-07:00,2023-07-06 16:08:30-07:00,LTD 4,0 days 00:57:30
9,San Francisco Caltrain Station,2023-07-06 15:11:00-07:00,2023-07-06 15:11:00-07:00,0 days,Sunnyvale Caltrain Station,2023-07-06 16:13:00-07:00,2023-07-06 16:13:45-07:00,LTD 4,0 days 01:02:45


In [82]:
departure_df = self.departures_response_to_next_trains_stopping_at_station(self.my_departure_station_id).rename(
    columns={
        "stop_name": "departure_stop",
        "time_late": "late_departing",
    }
)
arrival_df = self.departures_response_to_next_trains_stopping_at_station(self.my_destination_station_id).rename(
    columns={
        "stop_name": "arrival_stop",
        "time_late": "late_arriving",
        "scheduled_departure": "scheduled_arrival",
    }
)


In [85]:
self.next_train_options()

,departure stop,arrival stop,scheduled departure,late departing,scheduled arrival,late arriving,travel time
0,San Francisco Caltrain Station,Redwood City Caltrain Station,15:11,0 minutes,15:53,0 minutes,42 minutes
1,San Francisco Caltrain Station,Redwood City Caltrain Station,15:30,0 minutes,16:08,0 minutes,38 minutes


In [4]:
# self.fetch_data()
self.next_train_options()

,departure stop,arrival stop,scheduled departure,late departing,scheduled arrival,late arriving,travel time
0,San Francisco Caltrain Station,Redwood City Caltrain Station,08:27,0 minutes,09:07,0 minutes,40 minutes
1,San Francisco Caltrain Station,Redwood City Caltrain Station,08:39,0 minutes,09:32,0 minutes,53 minutes


In [17]:
HTML(
    self.next_train_options()
    .data.set_index("departure stop")
    .pipe(format_for_display)
    .to_html(index=True)
)

,arrival stop,scheduled departure,late departing,scheduled arrival,late arriving,travel time
departure stop,,,,,,
Redwood City Caltrain Station,San Francisco Caltrain Station,10:18,0 minutes,10:57,0 minutes,39 minutes
Redwood City Caltrain Station,San Francisco Caltrain Station,10:37,0 minutes,11:33,0 minutes,56 minutes


In [24]:
gm.get_messages(query="subject:(caltrain status)")

[Message(to: alex.hakso@gmail.com, from: alex.hakso@gmail.com, id: 188e9353bcdf4682)]

In [142]:
south.next_train_options()

,departure stop,arrival stop,scheduled departure,late departing,scheduled arrival,late arriving,travel time
0,San Francisco Caltrain Station,Redwood City Caltrain Station,16:39,0 minutes,17:30,0 minutes,51 minutes
1,San Francisco Caltrain Station,Redwood City Caltrain Station,17:06,0 minutes,17:41,0 minutes,35 minutes
2,San Francisco Caltrain Station,Redwood City Caltrain Station,17:12,0 minutes,17:54,0 minutes,42 minutes


In [135]:
# self.fetch_data()
north.get_next_sf_trips_from_rwc()

trains {'411', '309', '127', '709'} not in the realtime response :thunk:


,stop name,scheduled departure,expected departure,time late,stop name onward stop,scheduled departure onward stop,expected departure onward stop,trip duration from origin
14,Redwood City Caltrain Station,16:40,16:40,0 minutes,San Francisco Caltrain Station,17:32,17:32,52 minutes
15,Redwood City Caltrain Station,17:03,17:03,1 minutes,nan,-,-,-nan minutes
16,Redwood City Caltrain Station,17:17,17:17,1 minutes,nan,-,-,-nan minutes
17,Redwood City Caltrain Station,17:28,17:28,0 minutes,nan,-,-,-nan minutes
18,Redwood City Caltrain Station,17:40,17:40,0 minutes,nan,-,-,-nan minutes


# Misc info gathering

In [13]:
CALTRAIN_OPERATOR_ID = "CT"
# This gives caltrain operatorID: CT
operators = f"http://api.511.org/transit/operators?api_key={os.environ['CALTRAIN_API_KEY']}"
# this gives the name of rwc caltrain stop
stops = f"http://api.511.org/transit/stops?api_key={os.environ['CALTRAIN_API_KEY']}&operator_id={CALTRAIN_OPERATOR_ID}&format=json"

In [17]:
response_stops = RwcSfTrains.request_to_dict(stops)
# [x for x in response_stops.json()['Contents']['dataObjects']['ScheduledStopPoint'] if 'Redwood'.lower() in x['Name'].lower()]

In [69]:
self.all_rwc_trains_and_onward_stops()

trains {'703', '405', '301', '403'} not in the realtime response :thunk:


,stop name,scheduled departure,expected departure,time late,stop name onward stop,scheduled departure onward stop,expected departure onward stop,line type,trip duration from origin
0,Redwood City Caltrain Station,07:03,07:03,1 minutes,nan,-,-,nan,-nan minutes
1,Redwood City Caltrain Station,07:17,07:17,1 minutes,nan,-,-,nan,-nan minutes
2,Redwood City Caltrain Station,07:28,07:28,0 minutes,nan,-,-,nan,-nan minutes
3,Redwood City Caltrain Station,07:40,07:40,0 minutes,Hillsdale Caltrain Station,07:52,07:52,Local,12 minutes
4,Redwood City Caltrain Station,07:40,07:40,0 minutes,Hayward Park Caltrain Station,07:55,07:55,Local,15 minutes
5,Redwood City Caltrain Station,07:40,07:40,0 minutes,San Mateo Caltrain Station,07:58,07:58,Local,18 minutes
6,Redwood City Caltrain Station,07:40,07:40,0 minutes,Burlingame Caltrain Station,08:02,08:02,Local,22 minutes
7,Redwood City Caltrain Station,07:40,07:40,0 minutes,Millbrae Caltrain Station,08:06,08:06,Local,27 minutes
8,Redwood City Caltrain Station,07:40,07:40,0 minutes,San Bruno Caltrain Station,08:11,08:11,Local,31 minutes
9,Redwood City Caltrain Station,07:40,07:40,0 minutes,South San Francisco Caltrain Station,08:15,08:15,Local,35 minutes


In [51]:
self.real_time_response['Siri']['ServiceDelivery']['VehicleMonitoringDelivery']

{'version': '1.4',
 'ResponseTimestamp': '2023-06-26T14:47:17Z',
 'VehicleActivity': [{'RecordedAtTime': '2023-06-26T14:47:16Z',
   'ValidUntilTime': '',
   'MonitoredVehicleJourney': {'LineRef': 'L1',
    'DirectionRef': 'N',
    'FramedVehicleJourneyRef': {'DataFrameRef': '2023-06-26',
     'DatedVehicleJourneyRef': '107'},
    'PublishedLineName': 'Local',
    'OperatorRef': 'CT',
    'OriginRef': '70271',
    'OriginName': 'Tamien Caltrain Station',
    'DestinationRef': '70011',
    'DestinationName': 'San Francisco',
    'Monitored': True,
    'InCongestion': None,
    'VehicleLocation': {'Longitude': '-122.27375', 'Latitude': '37.5191612'},
    'Bearing': None,
    'Occupancy': None,
    'VehicleRef': '107',
    'MonitoredCall': {'StopPointRef': '70121',
     'StopPointName': 'Belmont Caltrain Station',
     'DestinationDisplay': 'San Francisco',
     'VehicleLocationAtStop': '',
     'VehicleAtStop': '',
     'AimedArrivalTime': '2023-06-26T14:48:00Z',
     'ExpectedArrivalTime